 # Póker Mind - Creación del Dataset

**Introducción**

Este módulo del proyecto Póker Mind tiene como objetivo construir un dataset con una gran cantidad de manos de póker mediante simulaciones basadas en técnicas de Monte Carlo. Este dataset nos permitirá próximamente poder generar insights estadísticos sobre las probabilidades de éxito de distintas manos en escenarios con múltiples rivales y cartas comunitarias.

El presente notebook abarca la etapa de la creación del conjunto de datos, en la que simulamos miles de partidas de póker, registramos las mejores manos posibles en cada fase del juego (flop, turn, river), y obtenemos el rendimiento relativo de las manos preflop en distintos contextos para posteriormente evaluarlos.

**Generación del Dataset**

El dataset fue construido utilizando simulaciones de partidas de póker generadas con las siguientes características:

- Número de simulaciones: Realizaremos 1 Millón de simulaciones de manos para obtener una ran precisión en nuestros resultados
- Cartas iniciales: Se generan 2 cartas para el jugador (mano preflop).
- Cartas comunitarias: Se generan 5 cartas (flop: 3 cartas, turn: 1 carta, river: 1 carta).
- Número de rivales: Varía aleatoriamente entre 1 y 8 jugadores por simulación.
- Clasificación preflop: Se evalúa si la mano es una pareja o carta alta, y si las cartas son suited (mismo palo) u offsuit (palos diferentes).
- Mejor mano final: Se selecciona la mejor combinación de 5 cartas posible para el jugador y cada rival.
- Resultado: Se determina si el jugador ganó, perdió, o empató contra los rivales.

El dataset se almacena en un archivo .csv para su posterior análisis y visualización.

**¿Cómo lo haremos?**

Para la construcción de este dataset vamos a utilizar el famoso *método de Montecarlo*. 

**Simulaciones Monte Carlo**

Las simulaciones de Monte Carlo son un enfoque probabilístico que permite aproximar soluciones a problemas complejos mediante simulaciones aleatorias repetidas. En este caso, utilizamos Monte Carlo para modelar:

- La distribución de cartas en el póker.
- La probabilidad de formar determinadas manos dadas las cartas comunitarias.
- La probabilidad de ganar contra diferentes rivales.

**Combinaciones en el póker**

En póker, las combinaciones posibles de manos se calculan usando el concepto de combinaciones en matemáticas. Para un deck estándar de 52 cartas, las combinaciones se definen como:


$$
C(n, k) = \frac{n!}{k!(n - k)!}
$$

Donde:

- 𝑛 es el total de cartas disponibles (52 del mazo de Póker).
- 𝑘 es el número de cartas seleccionadas que conforman una mano.

Ejemplo: El número total de combinaciones de 5 cartas seleccionadas de un deck de 52 es:

$$
C(52, 5) = \frac{52!}{5!(52 - 5)!} = 2,598,960
$$

Este principio se utiliza para calcular todas las manos posibles y determinar la mejor mano en cada etapa.

**Objetivo del análisis**

El objetivo principal de este análisis es explorar el dataset generado y obtener insights sobre el juego de póker. Queremos responder a preguntas como:

* ¿Cuáles son las manos más fuertes?
* ¿Cómo afectan las cartas iniciales a la probabilidad de ganar?
* ¿Qué importancia tienen las cartas comunitarias en el desarrollo de una mano?

### Importación de librerías

In [32]:
import random
from collections import Counter
from itertools import combinations
import pandas as pd
import os

### **Función `crear_deck()`**

Esta función genera un mazo estándar de cartas de Poker. El mazo contiene 52 cartas divididas en 4 palos (`♠`, `♥`, `♦`, `♣`) y 13 valores (`2`, `3`, `4`, ..., `K`, `A`).

In [33]:
def crear_deck():
    palos = ['♠', '♥', '♦', '♣']
    valores = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
    return [f'{v}{p}' for v in valores for p in palos]

### **Función `clasificar_mano(cartas)`**
Esta función clasifica una mano de 5 cartas en uno de los siguientes tipos de manos de poker, según las reglas estándar del juego:

- Escalera Real (Royal Flush)
- Escalera de Color (Straight Flush)
- Póker (Four of a Kind)
- Full House (Full House)
- Color (Flush)
- Escalera (Straight)
- Trío (Three of a Kind)
- Doble Pareja (Two Pair)
- Pareja (One Pair)
- Carta Alta (High Card)

In [34]:
def clasificar_mano(cartas):
    valores = [carta[:-1] for carta in cartas]  # Obtenemos sólo los valores de las cartas (sin los palos)
    palos = [carta[-1] for carta in cartas]  # Obtenemos los palos de las cartas

    # Definimos los valores en orden
    valores_ordenados = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
    
    # Frecuencia de los valores de las cartas
    conteo_valores = Counter(valores)
    conteo_palos = Counter(palos)

    # Verificaciones
    es_color = len(conteo_palos) == 1  # Color Todas las cartas del mismo palo
    valores_int = [valores_ordenados.index(v) for v in valores]
    valores_int.sort()

    # Escalera
    es_escalera = len(conteo_valores) == 5 and (max(valores_int) - min(valores_int)) == 4  # cinco valores consecutivos
    
    # Caso especial escalera A, 2, 3, 4, 5
    if set(valores) == {'A', '2', '3', '4', '5'}:
        es_escalera = True
        valores_int = [valores_ordenados.index('2'), valores_ordenados.index('3'), 
                       valores_ordenados.index('4'), valores_ordenados.index('5'), valores_ordenados.index('A')]
        valores_int.sort()

    # Evaluamos las manos
    if es_color and es_escalera:
        if valores_int == [8, 9, 10, 11, 12]:  # Escalera Real
            return (10, valores, "Escalera Real")  # 10 = Escalera Real
        return (9, valores, "Escalera de Color")  # 9 = Escalera de Color
    elif 4 in conteo_valores.values():
        return (8, valores, "Póker")  # 8 = Póker
    elif 3 in conteo_valores.values() and 2 in conteo_valores.values():
        return (7, valores, "Full House")  # 7 = Full House
    elif es_color:
        return (6, valores, "Color")  # 6 = Color
    elif es_escalera:
        return (5, valores, "Escalera")  # 5 = Escalera
    elif 3 in conteo_valores.values():
        return (4, valores, "Trío")  # 4 = Trío
    elif list(conteo_valores.values()).count(2) == 2:
        return (3, valores, "Doble Pareja")  # 3 = Doble Pareja
    elif 2 in conteo_valores.values():
        return (2, valores, "Pareja")  # 2 = Pareja
    else:
        return (1, valores, "Carta Alta")  # 1 = Carta Alta

### **Función `clasificar_mano_preflop(cartas)`**
Clasifica las dos cartas iniciales del jugador en manos preflop. Determina si son una **pareja** o **carta alta**, y si son **suited** (mismo palo) o **offsuit** (palos diferentes). Retorna una cadena que indica el tipo de mano y su categoría suited u offsuit.


In [35]:
def clasificar_mano_preflop(cartas):
    # Si las dos cartas tienen el mismo valor, es una pareja
    if cartas[0][:-1] == cartas[1][:-1]:
        tipo = "Pareja"
    else:
        tipo = "Carta Alta"
    
    # Comprobamos si son del mismo palo
    if cartas[0][-1] == cartas[1][-1]:
        suited = "Suited"
    else:
        suited = "Offsuit"
    
    return f"{tipo} {suited}"

### **Función `valor_carta(carta)`**
Esta función convierte el valor de una carta a su valor numérico correspondiente, utilizado para comparar cartas en el proceso de evaluación de manos.

In [36]:
def valor_carta(carta):
    """Convierte el valor de una carta a su valor numérico correspondiente."""
    valores_figuras = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10,
                       'J': 11, 'Q': 12, 'K': 13, 'A': 14}
    return valores_figuras[carta]

### **Función `obtener_mejor_mano(cartas)`**
Esta función evalúa todas las combinaciones posibles de 5 cartas a partir del conjunto dado de cartas. Para seleccionar la mejor mano, utiliza una subfunción interna llamada `evaluar_mano()`, que clasifica la mano y realiza un desempate basado en los valores de las cartas (kicker).

La fórmula utilizada para evaluar la mejor mano es:

$$
\text{Mejor Mano} = \max \left( \text{combinaciones}(\text{cartas}, 5), \text{key} = \text{evaluar\_mano} \right)
$$

Donde `evaluar_mano()` devuelve la clasificación de la mano junto con un desempate basado en los valores numéricos de las cartas.

Finalmente, retorna la clasificación de la mejor mano junto con las cartas correspondientes.



In [37]:
def obtener_mejor_mano(cartas):
    
    combinaciones_manos = combinations(cartas, 5)
    
    # Evaluamos una mano con desempate por kicker
    def evaluar_mano(c):
        clasificacion, valores, _ = clasificar_mano(c)
        valores_int = sorted([valor_carta(v) for v in valores], reverse=True)  # Ordenamos valores numéricos descendente
        return (clasificacion, valores_int)  # Clasificación principal y desempate por valores

    # Seleccionamos la mejor mano considerando clasificación y desempates
    mejor_mano = max(combinaciones_manos, key=evaluar_mano)
    return clasificar_mano(mejor_mano), mejor_mano

### **Función `comparar_manos(mano1, mano2)`**
Esta función compara dos manos de póker en base a su clasificación y a un desempate de valores cuando las clasificaciones son iguales.

**Pasos de la comparación:**

1. **Comparación de la clasificación principal:**
   - Si la clasificación de `mano1` es superior a la de `mano2`, se devuelve `1`.
   - Si la clasificación de `mano1` es inferior a la de `mano2`, se devuelve `-1`.
   - Si las clasificaciones son iguales, se procede al desempate.

2. **Desempate de valores:**
   - Si las manos son iguales en clasificación, se desempata utilizando los valores de las cartas.
   - Las manos se ordenan por los valores de las cartas de mayor a menor, y se comparan de manera secuencial.
   - En caso de empate, se revisan manos específicas como el **Full House**, donde se compara el trío y luego la pareja.

**Ejemplo de desempate en Full House:**
  - Se compara primero el valor del trío (el valor que aparece tres veces), y luego la pareja (el valor que aparece dos veces).
  
**Devuelve:**
- `1` si `mano1` es superior a `mano2`.
- `-1` si `mano1` es inferior a `mano2`.
- `0` si ambas manos son iguales.


In [38]:
def comparar_manos(mano1, mano2):

    # Comparamos clasificación principal
    if mano1[0] > mano2[0]:
        return 1
    elif mano1[0] < mano2[0]:
        return -1
    else:
        # En caso de clasificación igual, desempatar por valores
        valores_ordenados = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        valores1 = Counter(mano1[1])
        valores2 = Counter(mano2[1])
        
        # Si ambos son Full House, comparamos primero el trío y luego la pareja
        if mano1[0] == 7:  # Full House
            trio1 = max((v for v in valores1 if valores1[v] == 3), key=lambda x: valores_ordenados.index(x))
            trio2 = max((v for v in valores2 if valores2[v] == 3), key=lambda x: valores_ordenados.index(x))
            if valores_ordenados.index(trio1) > valores_ordenados.index(trio2):
                return 1
            elif valores_ordenados.index(trio1) < valores_ordenados.index(trio2):
                return -1
            else:
                pareja1 = max((v for v in valores1 if valores1[v] == 2), key=lambda x: valores_ordenados.index(x))
                pareja2 = max((v for v in valores2 if valores2[v] == 2), key=lambda x: valores_ordenados.index(x))
                if valores_ordenados.index(pareja1) > valores_ordenados.index(pareja2):
                    return 1
                elif valores_ordenados.index(pareja1) < valores_ordenados.index(pareja2):
                    return -1
                else:
                    return 0  # Full House idéntico

        # Para otras manos, desempatamos por los valores individuales
        valores1 = sorted([valor_carta(v) for v in mano1[1]], reverse=True)
        valores2 = sorted([valor_carta(v) for v in mano2[1]], reverse=True)
        for v1, v2 in zip(valores1, valores2):
            if v1 > v2:
                return 1
            elif v1 < v2:
                return -1
        return 0  # Totalmente igual devolvemos empate

### **Función `obtener_resultado(mejor_mano_jugador, manos_rivales_completas)`**
Compara la mejor mano del jugador con las manos de los rivales para determinar el resultado de la partida.

- **Derrota**: Si el jugador pierde contra algún rival.
- **Empate**: Si el jugador empata con algún rival pero no pierde.
- **Victoria**: Si el jugador no pierde ni empata con nadie.

Devuelve `"Victoria"`, `"Empate"` o `"Derrota"`.


In [39]:

def obtener_resultado(mejor_mano_jugador, manos_rivales_completas):
    derrota = False
    empate = False
    
    for mano_rival in manos_rivales_completas:
        comparacion = comparar_manos(mejor_mano_jugador, mano_rival)
        
        if comparacion == -1:  
            derrota = True
        elif comparacion == 0: 
            empate = True

    if derrota:
        return "Derrota"
    
    if empate:
        return "Empate"
    
    return "Victoria"

### **Función `simulacion_montecarlo(num_simulaciones=1)`**
Realiza simulaciones de partidas de póker utilizando el método de Monte Carlo.

- **Proceso**:
  1. Crea un mazo de cartas y reparte 2 cartas al jugador y 2 cartas a cada rival (1 a 8 rivales aleatorios).
  2. Simula las cartas comunitarias (flop, turn y river).
  3. Evalúa la mejor mano del jugador y de los rivales en cada fase del juego.
  4. Compara la mano final del jugador con las manos de los rivales y determina el resultado (Victoria, Empate o Derrota).
  5. Guarda los resultados de la simulación en un DataFrame y lo exporta como un archivo `.csv`.

- **Devuelve**: Un archivo `simulacion_poker.csv` con las simulaciones.


In [40]:
def simulacion_montecarlo(num_simulaciones=1):
    
    dataset = [] # Creamos el dataset vacio
    
    for _ in range(num_simulaciones):
        # Creamos el deck y repartimos cartas
        deck = crear_deck()
        cartas_jugador = random.sample(deck, 2)  # Dos cartas para el jugador
        deck = [card for card in deck if card not in cartas_jugador]  # Eliminamos las cartas del jugador del deck
        
        # Clasificamos la mano preflop del jugador
        nombre_mano_preflop = clasificar_mano_preflop(cartas_jugador)
        
        # Número de rivales
        n_rivales = random.randint(1, 8)
        
        # Manos de los rivales
        manos_rivales = []
        for _ in range(n_rivales):
            mano_rival = random.sample(deck, 2)
            manos_rivales.append(mano_rival)
            deck = [card for card in deck if card not in mano_rival]  # Eliminamos cartas del rival del deck
        
        # Cartas comunitarias (flop, turn, river)
        flop = random.sample(deck, 3)
        deck = [card for card in deck if card not in flop] 
        turn = random.sample(deck, 1)
        deck = [card for card in deck if card not in turn] 
        river = random.sample(deck, 1)
        
        # Evaluamos la mejor mano del jugador en cada fase
        mejor_mano_flop, cartas_mejor_flop = obtener_mejor_mano(cartas_jugador + flop)
        mejor_mano_turn, cartas_mejor_turn = obtener_mejor_mano(cartas_jugador + flop + turn)
        mejor_mano_river, cartas_mejor_river = obtener_mejor_mano(cartas_jugador + flop + turn + river)

        # Evaluamos las manos rivales con las 5 mejores cartas: 2 del rival + las comunitarias
        manos_completas_rivales = [obtener_mejor_mano(mano_rival + flop + turn + river)[0] for mano_rival in manos_rivales]

        # Obtenemos el resultado final comparando la mano del jugador con todas las manos rivales
        resultado = obtener_resultado(mejor_mano_river, manos_completas_rivales)

        # Guardamos la simulación
        fila = {
            'cartas_jugador': cartas_jugador,
            'num_rivales': n_rivales,
            'mano_preflop': nombre_mano_preflop,
            'flop': flop,
            'mano_flop': mejor_mano_flop[2],
            'cartas_flop': list(cartas_mejor_flop),
            'turn': turn,
            'mano_turn': mejor_mano_turn[2],
            'cartas_turn': list(cartas_mejor_turn),
            'river': river,
            'mano_river': mejor_mano_river[2],
            'cartas_river': list(cartas_mejor_river),
            'resultado': resultado
        }
        
        dataset.append(fila)

    # Convertimos a DataFrame y lo guardamos
    df = pd.DataFrame(dataset)
    output_path = os.path.join('data', 'simulacion_poker.csv')
    os.makedirs('data', exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"Dataset guardado en: {output_path}")

### **Simulación de 1,000,000 manos de póker**
Ejecutamos la función `simulacion_montecarlo(num_simulaciones=1000000)` para realizar 1,000,000 simulaciones de manos de póker. Este proceso genera el conjunto de datos con toda la información. El resultado lo guardamos en `data/simulacion_poker.csv` para su posterior análisis.


In [41]:
simulacion_montecarlo(num_simulaciones=1000000)

Dataset guardado en: data\simulacion_poker.csv
